# 05 - model2 -- nn - scoring
Note: this notebook was used to develop the script 'score_with_nn.py'

In [10]:
import numpy as np
import pandas as pd
import tensorflow as tf
import os
from tensorflow import keras
from sklearn.metrics import classification_report
import pickle
import time

In [11]:
model = tf.keras.models.load_model('/home/jupyter/sb-entity-classification/models/nn_embedding.h5')
end_to_end_model =  tf.keras.models.load_model('/home/jupyter/sb-entity-classification/models/end_to_end_nn_embedding')

In [12]:
with open("/home/jupyter/sb-entity-classification/models/class_names.txt", "rb") as fp:  
    class_names = pickle.load(fp)

def get_prediction(string_input):
    probabilities = end_to_end_model.predict([[string_input]])
    return class_names[np.argmax(probabilities[0])]

In [13]:
test = pd.read_pickle("/home/jupyter/sb-entity-classification/data/test_nn.pkl")
test = test.sample(5)
X_test = test['name'].tolist()

In [17]:
start = time.time()
preds = list(map(get_prediction,X_test))
time_taken = time.time() - start
avg_time_p = time_taken / len(X_test)
print('Avg time per prediction {:.2f}s'.format(avg_time_p))
print('Total time for all {} forecasts: {:.2f}h'.format(len(X_test), time_taken / 3600))

print(classification_report(test['class_name'], preds, digits=3))

Avg time per prediction 0.13s
Total time for all 5 forecasts: 0.00h
                        precision    recall  f1-score   support

                 Album      1.000     1.000     1.000         1
               Athlete      1.000     1.000     1.000         2
              Building      0.000     0.000     0.000         0
EducationalInstitution      0.000     0.000     0.000         1
               Village      1.000     1.000     1.000         1

              accuracy                          0.800         5
             macro avg      0.600     0.600     0.600         5
          weighted avg      0.800     0.800     0.800         5



/opt/conda/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1

In [19]:
with open('/home/jupyter/sb-entity-classification/data/test_preds.pkl', "wb") as fp:   
    pickle.dump(preds, fp)